## Word Embeddings
* Any feed-forward neural network that takes words from a vocabulary as input and embeds them as vectores into a lower-dimensional space, which it then fine-tunes through back-propagation, necessarily yields word embeddings as the weights of the first layer, which is usually referred to as the __Embedding Layer__.
* Instead of doing this as a by-product of training, , the generation of word embeddings is the explicit goal of this process.
* The hope is to produce word embeddings that encode meaningful relationships so that these embeddings can be used by other models, instead of being haphazardly or idiosyncratically linked to one specific model.

## word2vec: The skip-gram model
* We will train a simple neural network with a single hidden layer
* Basic idea: Predict context words using center word
<img src="../pics/word2vec_training_data.png">
* And once we've trained it, we're not going to use the model for the task it was trained on (to predict context words), but instead use the model's weights as "word vectors" which are (hopefully) more meaningful representations of the word's meaning.
* It is possible to use **negative sampling** as the training method. Negative sampling is a simplified version of Noise Contrastive Estimation that makes certain assumptions about the number of noise samples to generate (k) and the distribution of noise samples (Q) - namely that kQ(w) = 1
* TODO: bring in explanation
* But negative sampling doesn't have the theoretical guarantee that its derivative tends towards the gradient of the softmax function. 
* NCE, on the other hand, has nice theoretical guarantees.
* Whether its negative sampling or NCE, this is only useful at training time.
* During inference, the full softmax still needs to be computed to obtain a normalized probability.

### Noise Contrastive Estimation
#### Some issues learning word vectors using a "standard" neural network.
* Word vectors are learned while the network learns to predict the next word given a window of words (the input of the network).
* Predicting the next word is like predicting the class. 
* That is, such a network is just a "standard" multinomial (multi-class) classifier. 
* The problem is, the network must have as many output neurons as classes there are. 
* When the classes are actual words, the number of neurons is huge.
* A "standard" neural network is usually trained with a cross-entropy cost function which requires the values of the output neurons to represent probabilities - which means that the output "scores" computed by the network for each class have to be normalized and converted into actual probabilities for each class. 
* This normalization step is typically achieved using the softmax function, but softmax is very costly when applied to a huge output layer.

#### The Noise Contrastive Estimation (NCE) solution
* To deal with the expensive computation of the softmax, Word2Vec uses a technique called noise-contrastive estimation. 
* The basic idea is to convert a multinomial classification problem (predicting the next word is also multinomial classification) to a binary classification problem. 
* So, instead of using softmax to estimate a true probability distribution of the output word, a binary logistic regression (binary classification) is used instead.
* For each training sample, the optimized classifier is fed a true pair (a center word and another word that appears in its context) and a number of randomly corrupted pairs (consisting of the center word and a randomly chosen word from the vocabulary). 
* By learning to distinguish the true pairs from corrupted ones, the classifier ultimately learns the word vectors.
* This is important: instead of predicting the next word (the "standard" training technique), the optimized classifier simply predicts whether a pair of words is good or bad.
* Word2Vec slightly customizes the process and calls it negative sampling. 
* In Word2Vec, the words for the negative samples (used for the corrupted pairs) are drawn from a specially designed distribution, which favours less frequent words to be drawn more often.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow as tf

import word2vec_utils

# Model hyperparameters
VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128            # dimension of the word embedding vectors
SKIP_WINDOW = 1             # the context window
NUM_SAMPLED = 64            # number of negative examples to sample
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
VISUAL_FLD = 'visualization'
SKIP_STEP = 5000

# Parameters for downloading data
DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize

# Assemble The Graph
### Define placeholders for input and output
### Define the weight variables
### Define the inference model
### Define the loss function and optimizer

### Define placeholders
* Input is the center word
* Output is the target (context) word.
* Instead of using one-hot vectors, we input the index of those context words directly.
* Each sample input is a scalar, and the placeholder for BATCH_SIZE samples will thus have a shape of `[BATCH_SIZE]`.
* Note that our `center_words` and `target_words` are both being fed in as scalars - that is as their corresponding indices in our vocabulary.
* In this particular instance, we are feeding in a tensorflow `tf.Dataset` object, and then creating an iterator for it using `dataset.make_inializable_iterator()`

### Define the weights (in this case, embedding matrix)
* In word2vec, the weights are actually the main things that we care about.
* Each row corresponds to the representation vector for one word.
* For a given word represented by a vector of EMBED_SIZE size, the the embedding matrix will have a shape of [VOCAB_SIZE, EMBED_SIZE].
* The following code simply sets up the weight matrix for our word embeddings:

**`embed_matrix = tf.get_variable('embed_matrix', 
    shape=[VOCAB_SIZE, EMBED_SIZE], 
    initializer=tf.random_uniform_initializer())`**

* We initialize the embedding matrix with values from a random distribution - in this case we are using a uniform distribution.
* Initialization is done by using the `initializer` parameter in `tf.get_variable`.

### Define the inference model
* Our goal is to get the vector representations of words in our dictionary.
* Remember that the embed_matrix has dimension VOCAB_SIZE x EMBED_SIZE, with each row of the embedding matrix corresponding to the vector representation of the word at that index.
* So to get the representation of all the center words in the batch, we get a slice of all corresponding rows in the embedding matrix.
* TensorFlow provides a convenient method to do so called `tf.nn.embedding_lookup()`
* This method is really useful when it comes to matrix multiplication with one-hot vectors because it allows you to just do the computations that matter, avoiding the unneccessary multiplications with the zero entries in the one-hot vectors.
* So this line gives us the embedding (or vector representation) of the center words we input:
**`embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embedding')`

### Define the loss function and optimizer
* First we construct the variables for NCE loss as follows.
* These are the weights and biases for the hidden layer used to calculate the NCE loss.

`nce_weight = tf.get_variable('nce_weight', shape=[VOCAB_SIZE, EMBED_SIZE],
    initializer=tf.truncated_normal_initializer(stddev=1.0 / EMBED_SIZE ** 0.5)))`

`nce_bias = tf.get_variable('nce_bias', initializer=tf.zeros([VOCAB_SIZE]))`

* We then define our loss function using TensorFlow's built-in `tf.nn.nce_loss` function:

**`loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                            biases=nce_bias,
                            labels=target_words, 
                            inputs=embed, 
                            num_sampled=NUM_SAMPLED, 
                            num_classes=VOCAB_SIZE), name='loss')`**
                            
* Then for our optimizer we use gradient descent:

`optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)`

In [2]:
def word2vec(dataset):
    """ Build the graph for word2vec model and train it """
    # Step 1: get input, output from the dataset
    with tf.name_scope('data'):
        iterator = dataset.make_initializable_iterator()
        center_words, target_words = iterator.get_next()

    """ Step 2: define weights """
    with tf.name_scope('embed'):
        embed_matrix = tf.get_variable('embed_matrix', 
                                        shape=[VOCAB_SIZE, EMBED_SIZE],
                                        initializer=tf.random_uniform_initializer())
        
        
        """ Step 3: Inference (compute the forward path of the graph)"""
        embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embedding')

    # Step 4: construct variables for NCE loss and define loss function
    with tf.name_scope('loss'):
        nce_weight = tf.get_variable('nce_weight', shape=[VOCAB_SIZE, EMBED_SIZE],
                        initializer=tf.truncated_normal_initializer(stddev=1.0 / (EMBED_SIZE ** 0.5)))
        nce_bias = tf.get_variable('nce_bias', initializer=tf.zeros([VOCAB_SIZE]))

        # define loss function to be NCE loss function
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight, 
                                            biases=nce_bias, 
                                            labels=target_words, 
                                            inputs=embed, 
                                            num_sampled=NUM_SAMPLED, 
                                            num_classes=VOCAB_SIZE), name='loss')

    # Step 5: define optimizer
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
    
    word2vec_utils.safe_mkdir('logs/checkpoints')

    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer())

        total_loss = 0.0 # we use this to calculate late average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter('logs/graphs/word2vec_simple', sess.graph)

        for index in range(NUM_TRAIN_STEPS):
            try:
                loss_batch, _ = sess.run([loss, optimizer])
                total_loss += loss_batch
                if (index + 1) % SKIP_STEP == 0:
                    print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                    total_loss = 0.0
            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
        writer.close()

In [3]:
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

def main():
    dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), 
                                 tf.TensorShape([BATCH_SIZE, 1])))
    word2vec(dataset)

In [4]:
main()

Successfully downloaded data/text8.zip
Read tokens. There are 17005207 tokens
Average loss at step 4999:  65.2
Average loss at step 9999:  18.6
Average loss at step 14999:   9.6
Average loss at step 19999:   6.7
Average loss at step 24999:   5.6
Average loss at step 29999:   5.2
Average loss at step 34999:   5.0
Average loss at step 39999:   4.9
Average loss at step 44999:   4.8
Average loss at step 49999:   4.8
Average loss at step 54999:   4.7
Average loss at step 59999:   4.7
Average loss at step 64999:   4.7
Average loss at step 69999:   4.7
Average loss at step 74999:   4.6
Average loss at step 79999:   4.7
Average loss at step 84999:   4.7
Average loss at step 89999:   4.7
Average loss at step 94999:   4.6
Average loss at step 99999:   4.6
